In [11]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import pynsee as yns
import requests
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from cartiflette.s3 import download_vectorfile_url_all

import scipy as sp
import statistics
from sklearn import set_config
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import (r2_score,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             median_absolute_error,
                             max_error, make_scorer
                            )
from sklearn.metrics import PredictionErrorDisplay
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate,cross_val_score,validation_curve,learning_curve
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, OrdinalEncoder

set_config(display="diagram", print_changed_only=False)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Metriques de performances

In [2]:
def get_performances(real_value, estimated_value, metrics):
    perfs = []
    for metric_func in metrics:
        perfs.append((metric_func.__name__, metric_func(real_value, estimated_value)))
    return pd.DataFrame(perfs, columns=["metric", "value"])


def get_all_performances(value_train, values_test, metrics):
    test_perfs = []
    train_perfs = []
    metric_names = []
    for metric_func in metrics:
        metric_name = metric_func.__name__
        metric_names.append(metric_name)
        train_perfs.append(metric_func(*value_train))
        test_perfs.append(metric_func(*values_test))
    perfs = {"metric": metric_names, "train": train_perfs, "test": test_perfs,}
    return pd.DataFrame(perfs)

In [3]:
METRICS = [r2_score,
           mean_squared_error,
           median_absolute_error,
           max_error
          ]

In [14]:
base = gpd.read_file('df_velib.geojson')

In [15]:
base

INSEE_COM  cap_com  stat_com                       NOM  \
0      75101      710        26  Paris 1er Arrondissement   
1      75102      747        25   Paris 2e Arrondissement   
2      75103      437        15   Paris 3e Arrondissement   
3      75104      712        26   Paris 4e Arrondissement   
4      75105     1169        36   Paris 5e Arrondissement   
..       ...      ...       ...                       ...   
77     94069       83         3             Saint-Maurice   
78     94076      299        12                 Villejuif   
79     94080      306        10                 Vincennes   
80     94081      425        16           Vitry-sur-Seine   
81     95018      161         5                Argenteuil   

                       NOM_M           STATUT  POPULATION INSEE_DEP INSEE_REG  \
0   PARIS 1ER ARRONDISSEMENT  Capitale d'état       15917        75        11   
1    PARIS 2E ARRONDISSEMENT  Capitale d'état       21595        75        11   
2    PARIS 3E ARRONDISSEMENT  Capitale d'état       34025        75        11   
3    PARIS 4E ARRONDISSEMENT  Capitale d'état       29131        75        11   
4    PARIS 5E ARRONDISSEMENT  Capitale d'état       58227        75        11   
..                       ...              ...         ...       ...       ...   
77             SAINT-MAURICE   Commune simple       14189        94        11   
78                 VILLEJUIF   Commune simple       55208        94        11   
79                 VINCENNES   Commune simple       49788        94        11   
80           VITRY-SUR-SEINE   Commune simple       95510        94        11   
81                ARGENTEUIL  Sous-préfecture      111038        95        11   

         surface  \
0   1.828408e+06   
1   9.923087e+05   
2   1.170282e+06   
3   1.596122e+06   
4   2.543140e+06   
..           ...   
77  1.449035e+06   
78  5.282326e+06   
79  1.904540e+06   
80  1.166551e+07   
81  1.740858e+07   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
# from pynsee.download import download_file
# filosofi = download_file("FILOSOFI_COM_2016")

Les station vélib ne sont pas présentes dans toutes les communes équipées d'une piste cyclables, ce qui justifie la préence de valeurs manquantes pour les variables cap_com et stat_com. 2tant en présence de variables numériques positives, ces valeurs manquantes seront imputées par 0. 

In [31]:
base[['cap_com','stat_com']].fillna(0, inplace=True)

/tmp/ipykernel_15692/2220551359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base[['cap_com','stat_com']].fillna(0, inplace=True)


In [ ]:
Les variables nom_com, insee_com, source et geometry ne serons pas utilisées pour la modélisation

In [ ]:
data= base[['longueur','longueur_pond_hw','longueur_pond_adg','difference_hw','difference_adg','cap_com','stat_com','source	surface	population	geometry]]